In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from matplotlib import pyplot    
from PIL import Image
from skimage.io import imread, imsave

import zmq
import numpy as np
import cv2
import io
import timeit

In [2]:
model = load_model('generator_100.h5')

context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")

<SocketContext(bind='tcp://*:5555')>

In [3]:
# load an image
def load_image(pixels):
    # load image with the preferred size
    #pixels = load_img(filename, target_size=size)
    # convert to numpy array
    pixels = img_to_array(pixels)
    # scale from [0,255] to [-1,1]
    pixels = (pixels - 127.5) / 127.5
    # reshape to 1 sample
    pixels = np.expand_dims(pixels, 0)
    return pixels

In [ ]:
while True:
    bytes_received = socket.recv()
    print('Received sketch. Processing...')
    start = timeit.default_timer()
    image = Image.open(io.BytesIO(bytes_received))
    image = load_image(image)
    gen_image = model.predict(image)
    # scale from [-1,1] to [0,1]
    gen_image = (gen_image + 1) / 2.0    

    imsave('predicted_map.png', (gen_image[0] * 255.0).astype(np.uint8))
    predicted_map = imread('predicted_map.png')
    predicted_map = np.rot90(predicted_map, 3)
    
    bytes_to_send = predicted_map.tobytes()
    socket.send(bytes_to_send)
    stop = timeit.default_timer()
    print('Created heightmap. Time [s]: ', stop - start) 

Received sketch. Processing...
Created heightmap. Time [s]:  14.2172752
Received sketch. Processing...
Created heightmap. Time [s]:  0.6523542999999989
Received sketch. Processing...
Created heightmap. Time [s]:  0.6802380999999986
Received sketch. Processing...
Created heightmap. Time [s]:  0.6885072999999995
Received sketch. Processing...
Created heightmap. Time [s]:  0.673065900000001
Received sketch. Processing...
Created heightmap. Time [s]:  0.5684931999999989
Received sketch. Processing...
Created heightmap. Time [s]:  0.5899268000000006
Received sketch. Processing...
Created heightmap. Time [s]:  0.5959395000000001
Received sketch. Processing...
Created heightmap. Time [s]:  0.5870508000000001
Received sketch. Processing...
Created heightmap. Time [s]:  0.5006397999999948
Received sketch. Processing...
Created heightmap. Time [s]:  0.5011901000000023
Received sketch. Processing...
Created heightmap. Time [s]:  0.4447388999999973
Received sketch. Processing...
Created heightmap.

Created heightmap. Time [s]:  0.3173259999998663
Received sketch. Processing...
Created heightmap. Time [s]:  0.3482708999999886
Received sketch. Processing...
Created heightmap. Time [s]:  0.2982260999999653
Received sketch. Processing...
Created heightmap. Time [s]:  0.2990828000001784
Received sketch. Processing...
Created heightmap. Time [s]:  0.30420590000017
Received sketch. Processing...
Created heightmap. Time [s]:  0.30394090000004326
Received sketch. Processing...
Created heightmap. Time [s]:  0.2932846999999583
Received sketch. Processing...
Created heightmap. Time [s]:  0.29618630000004487
Received sketch. Processing...
Created heightmap. Time [s]:  0.2774910999999065
Received sketch. Processing...
Created heightmap. Time [s]:  0.2821025999999165
Received sketch. Processing...
Created heightmap. Time [s]:  0.27653989999998885
Received sketch. Processing...
Created heightmap. Time [s]:  0.2791666000000532
Received sketch. Processing...
Created heightmap. Time [s]:  0.2741628